In [96]:
import numpy as np
import pandas as pd
from scipy import interpolate as intp
from scipy import optimize
import matplotlib.pyplot as plt
import gsw
from tqdm import tqdm

In [110]:
def find_sigma0_z(salinity, temperature, pressure, latitude, longitude, sigmas):
    """
    Find the depth of the isopycnal using T/S from a cast
    Inputs:
        salinity    (array) : Salinity in PSU
        temperature (array) : In-situ temperature (C)
        latitude    (array) : Latitude of the sample
        longitude   (array) : Longitude of the sample
        sigmas      (array) : Isopycnals for which to find the depth
    Outputs:
        sigma_z     (array) : Depth of the isopycnals
    Algorithm:
        1. Convert in-situ salinity and temperature to Absolute Salinity and Conservative Temperature
        2. Calculate sigma0
        3. Check to ensure the isopycnal surface spans the density range of the water column
        4. Build interpolating functions for both T and S
    """    
    
    def dens_diff(pressure, SA_f, CT_f, target_sigma0):
        return np.square(gsw.sigma0(SA_f(pressure),CT_f(pressure)) - target_sigma0)
    
    def return_early(is_scalar):
        if is_scalar:
            return np.nan
        else:
            return np.nan*np.ones(sigmas.shape)
    
    # Promote to single element array if only one level requested
    valid = (~np.isnan(salinity)) & (~np.isnan(temperature)) & (~np.isnan(pressure))
    salinity = salinity[valid]
    temperature = temperature[valid]
    pressure = pressure[valid]
    longitude = longitude[valid]
    latitude = latitude[valid]    
        
    
    is_scalar = type(sigmas) == type(0.)
    if is_scalar:
        sigmas = np.array([sigmas])
        
    if valid.sum() < 3:
        return return_early(is_scalar)
    
    pressure = np.array(pressure)
    SA = gsw.SA_from_SP(salinity, pressure, longitude, latitude)
    CT = gsw.CT_from_t(salinity, temperature, pressure)
    
    P_sort = np.unique(pressure)    
    if len(P_sort) < 3:
        return return_early(is_scalar)
        
    SA_sort = np.zeros(P_sort.shape)
    CT_sort = np.zeros(P_sort.shape)        
    # Loop through all pressures that overlap, average the temperatures and salinity accordingly
    for idx, P in enumerate(P_sort):
        presidx = (pressure == P)        
        SA_sort[idx] = SA[presidx].mean()
        CT_sort[idx] = CT[presidx].mean()                
    
    try:
        SA_intp = intp.interp1d(P_sort, SA_sort, kind='quadratic')
        CT_intp = intp.interp1d(P_sort, CT_sort, kind='quadratic')
    except:
        print(SA_sort,CT_sort)

    sigma0 = gsw.sigma0(SA_sort,CT_sort)
    sigma0_max = sigma0.max()
    sigma0_min = sigma0.min()
    
    sigma0_z = np.zeros(len(sigmas))
    
    for sigidx, siglev in enumerate(sigmas):
        if (siglev < sigma0_min) or (siglev > sigma0_max):
            sigma0_z[sigidx] = np.nan
        else:
            for botidx in range(len(sigma0)-1):
                if (siglev >= sigma0[botidx]) & (siglev <= sigma0[botidx+1]):
                    start_idx = botidx
                    break
                                              
            out = optimize.minimize_scalar(dens_diff,
                                            bounds=[P_sort[botidx],P_sort[botidx+1]],
                                            method='Bounded',
                                            args=(SA_intp, CT_intp, siglev))
            
            sigma0_z[sigidx] = out.x

    if is_scalar:
        sigma0_z = np.squeeze(sigma0_z)
    
    return sigma0_z
    
    
    

In [81]:
path = '/home/ashao/data/glodap/'
path = ''
glodap = pd.read_csv(path+'GLODAPv2.2019_Merged_Master_File.csv')
cruise_ids = glodap.cruise.unique()
glodap_by_cruise = [ glodap[(glodap.cruise == cruise_id) & (glodap.salinityqc == 1)] for cruise_id in cruise_ids ]

In [82]:
cruise_df = glodap_by_cruise[0].replace(-9999,np.nan)

In [102]:
def process_cruise(cruise_df,sigma0_vals):
    nsigma0 = len(sigma0_vals)
    stations = cruise_df.station.unique()
    cols = ['latitude','longitude'] + [ f'{sig0}_z' for sig0 in sigma0_vals ]   
    df_out = pd.DataFrame(columns = cols)
    for station in stations:
        df = cruise_df[ cruise_df.station == station ]        
        sigma0_z = find_sigma0_z(df.salinity,df.temperature,df.pressure,df.latitude,df.longitude,sigma0_vals)
        if (np.isnan(sigma0_z).sum() != len(sigma0_vals)):
            out = np.nan*np.ones(nsigma0+2)
            out[0] = df.latitude.mean()
            out[1] = df.longitude.mean()
            out[2:] = sigma0_z        
            df_out = df_out.append( pd.DataFrame([out], columns = cols), ignore_index = True )
    
    return df_out
    

In [112]:
siglevels = np.linspace(26,27,11)
out = pd.concat([process_cruise(cruise_df.replace(-9999,np.nan),siglevels) for cruise_df in tqdm(glodap_by_cruise[:20])],ignore_index=True)









  0%|          | 0/20 [00:00<?, ?it/s]







  5%|▌         | 1/20 [00:00<00:12,  1.57it/s]







 10%|█         | 2/20 [00:01<00:11,  1.52it/s]







 15%|█▌        | 3/20 [00:01<00:08,  1.97it/s]







 20%|██        | 4/20 [00:01<00:07,  2.06it/s]







 25%|██▌       | 5/20 [00:02<00:06,  2.22it/s]







 30%|███       | 6/20 [00:03<00:08,  1.60it/s]







 35%|███▌      | 7/20 [00:04<00:08,  1.54it/s]







 40%|████      | 8/20 [00:04<00:07,  1.69it/s]







 45%|████▌     | 9/20 [00:05<00:08,  1.30it/s]







 55%|█████▌    | 11/20 [00:06<00:05,  1.60it/s]







 60%|██████    | 12/20 [00:07<00:06,  1.22it/s]







 65%|██████▌   | 13/20 [00:08<00:06,  1.14it/s]







 70%|███████   | 14/20 [00:09<00:04,  1.25it/s]







 75%|███████▌  | 15/20 [00:10<00:04,  1.19it/s]







 80%|████████  | 16/20 [00:10<00:02,  1.49it/s]







 85%|████████▌ | 17/20 [00:13<00:04,  1.37s/it]







 90%|█████████ | 18/20 [00:13<00:02,  1.12s/it]







 95%|█████████▌| 19/

In [113]:
out

,latitude,longitude,26.0_z,26.1_z,26.2_z,26.3_z,26.4_z,26.5_z,26.6_z,26.7_z,26.8_z,26.9_z,27.0_z
0,80.567,7.2267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.694537,13.583451,16.606930
1,80.633,9.4600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.971914,8.439523,11.051453,13.836842
2,80.733,12.8530,NaN,NaN,5.121509,7.155966,9.244169,11.388475,13.591835,15.858033,18.192005,20.600247,23.091437
3,81.052,17.6680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.996887,9.413883
4,81.197,16.7930,4.031728,5.014592,6.027652,7.073494,8.155112,9.275983,10.440211,11.652656,12.919195,14.247009,15.645048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,-51.946,-0.0037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.865204
433,-52.470,-0.0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.939342
434,-64.945,-41.5070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.531978,13.077785,14.689943,16.377679
435,-64.713,-43.3300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.558563,18.869654


In [94]:
pd.concat?

Signature:
pd.concat(
    objs,
    axis=0,
    join='outer',
    join_axes=None,
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    sort=None,
    copy=True,
)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis,
which may be useful if the labels are the same (or overlapping) on
the passed axis number.

Parameters
----------
objs : a sequence or mapping of Series or DataFrame objects
    If a dict is passed, the sorted keys will be used as the `keys`
    argument, unless it is passed, in which case the values will be
    selected (see below). Any None objects will be dropped silently unless
    they are all None in which case a ValueError will be raised.
axis : {0/'index', 1/'columns'}, default 0
    The axis to concatenate along.
join : {'inner', 'outer'}, default 'outer'
    How to handle indexes on othe

In [56]:
len(out2['27.0_z'])

164

In [74]:
glodap_by_cruise[1].replace(-9999,np.nan).sigma0.min()

27.467

In [70]:
out2

,latitude,longitude,26.0_z,26.1_z,26.2_z,26.3_z,26.4_z,26.5_z,26.6_z,26.7_z,26.8_z,26.9_z,27.0_z
0,-58.483,-0.990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-59.000,-1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-59.505,-1.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-60.514,-1.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-60.962,-1.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,-61.508,0.365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,-60.000,0.367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,-59.493,0.408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,-59.007,0.372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
